# Arguments

In [4]:
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

warmup_model_path = 'model-warmup.h5'
model_save_path = 'model-final.h5'
checkpoint_path = 'checkpoints-finetuning.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# --------------------------------------------------
num_of_epochs = 120
start_epoch = 0
# --------------------------------------------------


# Select training classes

In [5]:
class_paths_training = ['../datasets/plantvillage/___Late_blight', '../datasets/plantvillage/___Early_blight', '../datasets/plantvillage/___Septoria_leaf_spot', '../datasets/plantvillage/___Bacterial_spot']

# Load dataset

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.resize_image_preprocessor import resizeImagePreprocessor
from utils.preprocessors.img_to_array_preprocessor import ImgToArrayPreprocessor
from utils.io.dataset_loader import DatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

def load_datasets(path_list):
    # Load image paths
    image_paths = []
    print("[INFO] loading images...")
    for path in path_list:
        image_paths.extend(list(paths.list_images(path)))
        
    # Get unique classnames

    class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
    class_names = [str(x) for x in np.unique(class_names)]

    # Initial image preprocessing
    aap = resizeImagePreprocessor(input_width, input_height)
    iap= ImgToArrayPreprocessor()

    #Load image data and perform image data preprocessing
    dl = DatasetLoader(preprocessors=[aap,iap])
    (data,labels)  = dl.load(image_paths,verbose=500)
    data = data.astype("float") / 255.0


    # train test split
    (train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

    # convert the labels from integers to vectors
    train_y = LabelBinarizer().fit_transform(train_y)
    test_y = LabelBinarizer().fit_transform(test_y)
    
    return (train_x,test_x,train_y,test_y, class_names)

(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_training)



Using TensorFlow backend.


[INFO] loading images...
[INFO]: Processed 500/6807
[INFO]: Processed 1000/6807
[INFO]: Processed 1500/6807
[INFO]: Processed 2000/6807
[INFO]: Processed 2500/6807
[INFO]: Processed 3000/6807
[INFO]: Processed 3500/6807
[INFO]: Processed 4000/6807
[INFO]: Processed 4500/6807
[INFO]: Processed 5000/6807
[INFO]: Processed 5500/6807
[INFO]: Processed 6000/6807
[INFO]: Processed 6500/6807


In [7]:
print(len(class_names))
print(class_names)

4
['___Bacterial_spot', '___Early_blight', '___Late_blight', '___Septoria_leaf_spot']


# Load model from disk

In [8]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)

# Print index of layers

In [9]:
# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {:5}\t{:30}{}".format(i, layer.name, layer.__class__.__name__))

[INFO]     0	input_1                       InputLayer
[INFO]     1	block1_conv1                  Conv2D
[INFO]     2	block1_conv2                  Conv2D
[INFO]     3	block1_pool                   MaxPooling2D
[INFO]     4	block2_conv1                  Conv2D
[INFO]     5	block2_conv2                  Conv2D
[INFO]     6	block2_pool                   MaxPooling2D
[INFO]     7	block3_conv1                  Conv2D
[INFO]     8	block3_conv2                  Conv2D
[INFO]     9	block3_conv3                  Conv2D
[INFO]    10	block3_pool                   MaxPooling2D
[INFO]    11	block4_conv1                  Conv2D
[INFO]    12	block4_conv2                  Conv2D
[INFO]    13	block4_conv3                  Conv2D
[INFO]    14	block4_pool                   MaxPooling2D
[INFO]    15	block5_conv1                  Conv2D
[INFO]    16	block5_conv2                  Conv2D
[INFO]    17	dropout_2                     Dropout
[INFO]    18	block5_conv3                  Conv2D
[INFO]    19	dropout_

# Unfreeze final CONV layers

In [10]:
for layer in model.layers[:15]:
   layer.trainable = False
for layer in model.layers[15:]:
   layer.trainable = True
print('unfrozen')

unfrozen


# Compile model

In [11]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.0001, momentum=0.09)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


[INFO] re-compiling model ...


# Checkpoints

In [12]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [13]:
#load checkpoints if existing

import os

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)

# Training Monitor

In [14]:
from utils.callbacks.monitor import Monitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

callbacks.append(Monitor(fig_path, json_path, start_epoch))

# Fit model

In [ ]:
num_of_epochs = 100

H = model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/100
159/159 [==============================] - 534s 3s/step - loss: 0.6057 - accuracy: 0.7465 - val_loss: 0.5568 - val_accuracy: 0.7826

Epoch 00001: val_loss improved from inf to 0.55682, saving model to checkpoints-finetuning.hdf5
Epoch 2/100
159/159 [==============================] - 532s 3s/step - loss: 0.5885 - accuracy: 0.7556 - val_loss: 0.5328 - val_accuracy: 0.7703

Epoch 00002: val_loss improved from 0.55682 to 0.53281, saving model to checkpoints-finetuning.hdf5
Epoch 3/100
159/159 [==============================] - 533s 3s/step - loss: 0.5823 - accuracy: 0.7613 - val_loss: 0.5759 - val_accuracy: 0.7879

Epoch 00003: val_loss did not improve from 0.53281
Epoch 4/100
159/159 [==============================] - 534s 3s/step - loss: 0.5736 - accuracy: 0.7552 - val_loss: 0.5377 - val_accuracy: 0.8020

Epoch 00004: val_loss did not improve from 0.53281
Epoch 5/100
159/159 [==============================] - 534s 3s/step - loss: 0.5621 - accuracy: 0.7692 - val_loss: 0.5143 -

# Save model

In [ ]:
model.save(model_save_path)

# Evaluate with 20%

In [ ]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

# Evaluate with validation images

In [ ]:
class_paths_validation = ['../datasets/validation/___Early_blight', '../datasets/validation/___Appids', '../datasets/validation/___Leaf_miner', '../datasets/self/___Curly_top_virus']

In [ ]:
(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_validation)

from sklearn.metrics import classification_report

print("[INFO] evaluating with validation set...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))